## Importing OpenAI

In [1]:
import openai
import os

api_key ="API key in here"

os.environ["OPENAI_API_KEY"] = api_key
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


## Labeling task: instruction-based

In [6]:

import pandas as pd
import random as rd

data = pd.read_csv("Dataset/SignleFR_questions3.csv")

data_p_df = pd.DataFrame(data)
data_p_df = data_p_df.fillna('""')

### first section of the prompot
       
sec1 = f"""If the functional roles as lists of "Measure (M)",\
"Condition (C)", "Support (S)", "Spatial extent (SE)", and\
"Temporal extent (TE)". Also, Spatial relation (SR) can present\
in the functional roles of C, S, SE, they are denoted as SRc, SRs,\
SRse, respectively. Also, Temporal Relation (TR) can present in the\
functional roles of C, S, TE, they are denoted as TRc, TRs, TRte,\
respectively. Also, Time (T) or Place (P) can only be present in the\
functional role of M, they are denoted as Tm and Pm, respectively.\
Also, SR in here is based on 'Cardinal Direction Calculus (CDC)' or\
'Region Connection Calculus (RCC)'. Also, TR is based on 'Allen Interval Algebra (AIA)'."""
    
sec5 = ''
sec4 = ''
sec6 = ''
sec7 = ''
### second section of the prompt: Choosing one sample query has both SE and TE 
pattern = data_p_df.iloc[31]   
pattern = pattern.fillna('""')

main_query = pattern['Reformulated Question']


sec2 = "In the question, '{0}', the functional roles are identified as below:\
    \n- M: {1}\n- C: {2}\n- S: {3}\n- SE: {4}\n- TE {5}\
    \nAlso, the spatial and temporal relations in these roles are identified in the given question as below:\
    \n- Tm: {6}\n- Pm: {7}\n- SRc: {8}\n- TRc: {9}\n- SRs: {10}\n- TRs: {11}\n- SRse: {12}\n- TRte: {13}".format(
        main_query, pattern['M'],\
        pattern['C'], pattern['S'],\
        pattern['SE'], pattern['TE'],\
        pattern['Tm'], pattern['Pm'],\
        pattern['SRc'], pattern['TRc'],\
        pattern['SRs'], pattern['TRs'],\
        pattern['SRse'], pattern['TRte'])

  
    
### Third section of the prompt: instruct based on training set
prompt = []
training = [data.loc[i] for i in range(len(data_p_df)) if data_p_df.loc[i]['train-test'] == 'train']
training_df = pd.DataFrame(training)

for index, row in training_df.iterrows():
    
    Notnan_columns = [i for i in row[~row.isna()].index.tolist() if i !='Reformulated Question' \
                          and i !='Pattern' and i != 'Class'and i != 'train-test']
    sec3 = ''

    for i in Notnan_columns:
        sec3 += "{0}: {1}\n".format(i, row[i])

    sec4 += "In the question, '{0}', the functional roles are as below:\n{1}\n".format(row['Reformulated Question'],sec3)

sec5 = "{0}\n\n{1}\n\n{2}".format(sec1, sec2, sec4)
    
### fourth section of the prompt: ask to label based on test set

test = [data.loc[i] for i in range(len(data_p_df)) if data_p_df.loc[i]['train-test'] == 'test']
test_df = pd.DataFrame(test)
response_test = []
for index, row in test_df.iterrows():
 
    sec6 = "In the question, '{0}', the functional roles are as below:\n".format(row['Reformulated Question'])
    sec7 = sec5 + sec6 
    response = get_completion(sec7)
    response_test.append(response)


## Labeling task: Zero-shot Learning

In [ ]:
prompt_completion[0]['prompt']
    
# prompt = []
# data_p = [data.loc[i] for i in range(len(data)) if data.loc[i]['Pattern'] == pattern]
# data_p_df = pd.DataFrame(data_p)

test = []
for index, row in enumerate(prompt_completion):
    
    prompt = "Extract 'Measure', 'Support', 'Condition', 'Spatial extent', and 'Temporal extent' from\
    the below question:\n\
    '{0}'\n\
    If there is a spatial relation in the extracted 'Support', identify that and put it under 'Support_SR' label.\
    \nIf there is a temporal relation in the extracted 'Support', identify that and put it under 'Support_TR' label.\
    \nIf there is a spatial relation in the extracted 'Condition', identify that and put it under 'Condition_SR' label.\
    \nIf there is a temporal relation in the extracted 'Condition', identify that and put it under 'Condition_TR' label.\
    \nIf there is a spatial relation in the extracted 'Spatial extent', identify that and put it under 'SE_SR' label.\
    \nIf there is a temporal relation in the extracted 'Temporal extent', identify that and put it under 'TE_TR' label.\
    \nIn the cases where there are nothing found for each label, put ''.\
    \nSo, order the outputs as below in JSON format:\n\
    - Measure: measure\n\
    - Support: support\n\
    - Condition: condition\n\
    - Spatial extent: spatial_extent\n\
    -Temporal extent: temporal_extent\n\
    - Support_SR: support_sr\n\
    - Support_TR: support_tr\n\
    - Condition_SR: condition_sr\n\
    - Conidition_TR: condition_tr\n\
    - SE_SR: se_sr\n\
    - TE_TR: te_tr\n\
    ".format(row['prompt'])
    
    
    test.append(get_completion(prompt))
    

## Labeling task: Few-shot learning

In [ ]:
import pandas as pd
import numpy as np
import random
import json
dir_data = '/Users/mkazemi/cloudstor/2023/Visiting Research at UU/Simon Group/Paper/Dataset/'
fr_dataset = pd.read_csv(dir_data + "train_fr.csv")
fr_dataset = fr_dataset.fillna('')
# fr_dataset.head()

# fr_dataset.iloc[0]['Reformulated Question']


prompt_completion = []
for index, i in fr_dataset.iterrows():
    
    a = {"prompt":  i['Reformulated Question'] + "\n\n###\n\n",\
            "completion":"Measure:" + str([i['Measure']])  +\
            "\nCondition:" + str([i['Condition']]) +\
                "\nSupport:" + str([i['Support']]) +\
                    "\nSpatial_Extent:" + str([i['Spatial Extent']]) +\
                        "\nTemporal_Extent:" + str([i['Temporal extent']]) +\
                            "\nTime_measure:" + str([i['Time_measure']])+\
                             "\nPlace_measure:" + str([i['Place_measure']])+\
                                "\nSpatialRelation_Condition:" + str([i['Spatial Relation_Condition']])+\
                                 "\nTemporalRelation_Condition:" + str([i['Temporal Relation_Condition']])+\
                                "\nSpatialRelation_Support:" + str([i['Spatial Relation_Support']])+\
                                "\nTemporalRelation_Support:" + str([i['Temporal Relation_Support']])+\
                                "\nSpatialRelation_SpatialExtent:" + str([i['Spatial Relation_Spatial Extent']])+\
                                "\nTemporalRelation_TemporalExtent:" + str([i['Temporal Relation_Temporal Extent']]) + "\n END"}
    
    prompt_completion.append(a)

trainFR_gpt = [i for index,i in enumerate(prompt_completion) if index in [0,3,4,11,17,20,23,28,31,35]]
testFR_gpt = [i for i in prompt_completion if i not in trainFR_gpt]

file_name = "/Users/mkazemi/cloudstor/2023/Visiting Research at UU/Simon Group/Paper/Dataset/trainFR_gpt.jsonl"

with open(dir_data + 'trainFR_gpt.jsonl', "w") as output_file:
  for entry in trainFR_gpt:
    json.dump(entry, output_file)
    output_file.write("\n")
    
with open(dir_data + 'testFR_gpt.jsonl', "w") as output_file:
  for entry in testFR_gpt:
    json.dump(entry, output_file)
    output_file.write("\n")

completion_fr = []
for i in testFR_gpt:
    
    response = openai.Completion().create(
    
    model = 'curie:ft-home-2023-09-29-14-49-11',
    prompt = i['prompt'],
    max_tokens = 200,
    temperature = 0,
    # top_p =1,
    # frequency_penalty = 0,
    # presence_penalty = 0,
    stop = ['END']
    )
    
    completion_fr.append(response['choices'][0]['text'])
    

## Evaluation

In [5]:
# true_m_positive : the number of cases where GPT predicts a role which is the same with its true_m role.
# false_positive: the number of cases where GPT predicts is not the same with true_m role (true_m role existed, 
# but model indetifies a null or different part of senetence )
# false_negative: the number of cases where GPT predicts is not the same with true_m role (true_m role not existed, 
# but model indetifies a different part of senetence )
# true_m_negative: the number of cases where GPT predicted_m nothing and there is a no true_m role in benchmark as well.



tp = []
fp = []
fn = []
tn = []


    
entities = {'Measure':0, 'Condition':1, 'Support':2, 'Spatial_Extent':3, 'Temporal_Extent':4,\
               'Time_measure':5, 'Place_measure':6, 'SpatialRelation_Condition':7,'TemporalRelation_Condition':8,\
               'SpatialRelation_Support':9,'TemporalRelation_Support':10, 'SpatialRelation_SpatialExtent':11,\
               'TemporalRelation_TemporalExtent':12}



for index, m in enumerate(completion_fr):
    
        predicted = eval(m.split("\n")[0].split(":")[1])[0].rstrip()
        true= eval(testFR_gpt[index]['completion'].split("\n")[0].split(":")[1])[0]
    
        if predicted in true and true != '' and predicted != '': 
            tp.append((predicted, true))
        elif predicted not in true and true !='' and predicted != '' or true !='' and predicted == '':
            fp.append((predicted, true))
        
        elif predicted not in true and true =='' and predicted != '':
            fn.append((predicted, true))
    
        else:
            tn.append((predicted, true))
   
accuracy = (len(tp) + len(tn))/ (len(tp) + len(fp) + len(fn) + len(tn))
precision = (len(tp))/(len(tp) + len(fp))
recall = (len(tp))/(len(tp) + len(fn))
f1 = (2* recall_se* precision_se)/(precision_se + recall_se)
    
print("The evaluation metrics for are as below:\n\
            accuracy: {0}\n\
            precision: {1}\n\
            recall: {2}\n\
            f1: {3}".format(accuracy,precision,recall,f1))

# print(accuracy)
# print(precision)
# print(recall)
# print(f1)



## Generated Questions With Instruction

In [3]:

import pandas as pd
import random as rd

data = pd.read_csv("Dataset/SignleFR_questions2.csv")


def instructed_prompt(pattern, number_of_queries):
    
    prompt = []
    data_p = [data.loc[i] for i in range(len(data)) if data.loc[i]['Pattern'] == pattern]
    data_p_df = pd.DataFrame(data_p)

    
    ### first section of the prompot
       
    sec1 = f"""If the functional roles as lists of "Measure (M)", "Condition (C)", "Support (S)", "Spatial extent (SE)", and "Temporal extent (TE)". Also, Spatial relation (SR) can present in the functional roles of C, S, SE, they are denoted as SRc, SRs, SRse, respectively. Also, Temporal Relation (TR) can present in the functional roles of C, S, TE, they are denoted as TRc, TRs, TRte, respectively. Also, Time (T) or Place (P) can only be present in the functional role of M, they are denoted as Tm and Pm, respectively. Also, SR in here is based on 'Cardinal Direction Calculus (CDC)' or 'Region Connection Calculus (RCC)'. Also, TR is based on 'Allen Interval Algebra (AIA)'."""
    

    ### second section of the prompt: Choosing one sample query has both SE and TE 
        
        
    first_shot_class = [data_p_df.iloc[i] for i in range(len(data_p_df)) if data_p_df.iloc[i]['Class'] == pattern]
    first_shot_class_df = pd.DataFrame(first_shot_class[0])
    
    first_shot_class_df_na = first_shot_class_df.fillna('""')
    
    sec2 = "In the question, '{0}', the functional roles are identified as below:\
    \n- M: {1}\n- C: {2}\n- S: {3}\n- SE: {4}\n- TE {5}\
    \nAlso, the spatial and temporal relations in these roles are identified in the given question as below:\
    \n- Tm: {6}\n- Pm: {7}\n- SRc: {8}\n- TRc: {9}\n- SRs: {10}\n- TRs: {11}\n- SRse: {12}\n- TRte: {13}".format(
        list(first_shot_class_df_na.loc['Reformulated Question'])[0], list(first_shot_class_df_na.loc['M'])[0],\
        list(first_shot_class_df_na.loc['C'])[0], list(first_shot_class_df_na.loc['S'])[0],\
        list(first_shot_class_df_na.loc['SE'])[0], list(first_shot_class_df_na.loc['TE'])[0],\
        list(first_shot_class_df_na.loc['Tm'])[0], list(first_shot_class_df_na.loc['Pm'])[0],\
        list(first_shot_class_df_na.loc['SRc'])[0], list(first_shot_class_df_na.loc['TRc'])[0],\
        list(first_shot_class_df_na.loc['SRs'])[0], list(first_shot_class_df_na.loc['TRs'])[0],\
        list(first_shot_class_df_na.loc['SRse'])[0], list(first_shot_class_df_na.loc['TRte'])[0])
    
    
    
    ### Third section of the prompt: Few shot learning
    prompt = []
    
    for index, row in data_p_df.iterrows():
    
    
        Notnan_columns = [i for i in row[~row.isna()].index.tolist() if i !='Reformulated Question' \
                          and i !='Pattern' and i != 'Class']
        
        Notnan_columns_format = "'{0}'".format(', '.join(map(str, Notnan_columns)).replace(', ', "','"))
        
        sec3 = "Generate a new 'What' question that has only {0}:".format(Notnan_columns_format)
    
    
        if row['Class'] != pattern:
        
            sec5 =''
            for i in Notnan_columns:
            
                sec5 += "- {0}: {1}\n".format(i, row[i])
        
                sec4 = "{0}\nQuestion: {1}\n{2}\nGenerate {3} new 'What' questions that has only {4}:".format(
                    sec3, row['Reformulated Question'], sec5, number_of_queries, Notnan_columns_format)
            

            prompt.append("{0}\n\n{1}\n\n{2}".format(sec1, sec2, sec4))

    return prompt

prompt = instructed_prompt('M_C_S_SE_TE_SRse_TRte',2)[11]

response = get_completion(prompt)
